In [1]:
import pandas as pd
import numpy as np
import os
import glob
# import rdkit
import sklearn
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import f1_score


/opt/anaconda3/envs/my-rdkit-env/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Logistic regression and top 50 predictive descriptors


In [2]:
import multiprocessing as mp
# mp.set_start_method('forkserver',force=True)

import random
import numpy as np
import pandas as pd
# import pymc as pm
# import aesara.tensor as T
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
# import arviz as az
tox21_two_assays = pd.read_csv("ar_mda_ag+anta_with_descriptor_ina+a.csv")

one_assay = tox21_two_assays[tox21_two_assays['PROTOCOL_NAME'] == 'tox21-ar-mda-kb2-luc-agonist-p3']
one_assay.head()


X = one_assay.iloc[:,3:]
X_bayes = one_assay[['FpDensityMorgan2', 'BCUT2D_LOGPLOW','FractionCSP3']]
X_try = one_assay.iloc[:, 3:100]

y = one_assay.CHANNEL_OUTCOME


In [3]:
mapping = {'active antagonist': 1, 'inactive': 0}

y = y.replace(to_replace=['active antagonist', 'inactive'], value=[1, 0])

y.value_counts()


0    7845
1     984
Name: CHANNEL_OUTCOME, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

lr = LogisticRegression(random_state=42)

pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train, y_train)  # apply scaling on training data
# Pipeline(steps=[('standardscaler', StandardScaler()),
#                 ('logisticregression', LogisticRegression())])

pipe.score(X_test, y_test)

# fit the model with data
# lr.fit(X_train,y_train)

# #
# y_pred=lr.predict(X_test)

/opt/anaconda3/envs/my-rdkit-env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9150622876557192

In [5]:
y.value_counts()/one_assay.shape[0]

0    0.888549
1    0.111451
Name: CHANNEL_OUTCOME, dtype: float64

In [6]:
X_train.describe()
X_train.dtypes.value_counts()

float64    102
int64      102
dtype: int64

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,LabelBinarizer

numeric_columns=list(X_train.select_dtypes(['float64', 'int64']).columns)
categorical_columns=list(X_train.select_dtypes('object').columns)
print(numeric_columns)

pipeline=ColumnTransformer([
    ('num',StandardScaler(),numeric_columns),
    ('cat', 'passthrough', categorical_columns)])

new_X_train=pipeline.fit_transform(X_train)
new_X_train

['MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex', 'MinAbsEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA9', 'SlogP_VSA1', 'SlogP_VSA10', 'SlogP_VSA11', 'SlogP_VSA12', '

array([[ 0.11832558,  0.22485342,  0.11772358, ..., -0.08570828,
        -0.29192312, -0.15740412],
       [-1.83907602,  1.17931454, -1.83940427, ..., -0.08570828,
        -0.29192312, -0.15740412],
       [ 0.49924325, -1.96025458,  0.49858797, ..., -0.08570828,
         1.88482021, -0.15740412],
       ...,
       [-2.27296945,  0.86157709, -2.27323702, ..., -0.08570828,
        -0.29192312, -0.15740412],
       [ 0.39552164, -0.32452234,  0.39488087, ..., -0.08570828,
        -0.29192312, -0.15740412],
       [-0.52771827,  0.41461192, -0.52822993, ..., -0.08570828,
        -0.29192312, -0.15740412]])

In [8]:
X_scaled_test = pipeline.transform(X_test)
lg2 = LogisticRegression(class_weight = 'balanced', penalty='l1', solver = 'saga')
# fit it
lg2.fit(new_X_train,y_train)
# test
y_pred = lg2.predict(X_scaled_test)
yproba = lg2.predict_proba(X_scaled_test)
# performance
print(f'Accuracy Score: {accuracy_score(y_test,y_pred)}')
print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred)}')
print(f'R^2 square: {sklearn.metrics.r2_score(y_test,y_pred)}')

Accuracy Score: 0.8203095507738769
Confusion Matrix: 
[[1958  395]
 [  81  215]]
Area Under Curve: 0.779240274060716
Recall score: 0.7263513513513513
R^2 square: -0.8104030507345428


/opt/anaconda3/envs/my-rdkit-env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [9]:
yproba

array([[0.09555893, 0.90444107],
       [0.65613376, 0.34386624],
       [0.43435837, 0.56564163],
       ...,
       [0.98967956, 0.01032044],
       [0.11111646, 0.88888354],
       [0.55146857, 0.44853143]])

In [10]:
fea_coef = pd.DataFrame(lg2.coef_).T
fea_coef['features'] = X.columns
fea_coef = fea_coef.rename(columns = {0:'coef'})
top50fea = list(fea_coef.sort_values(by=['coef']).tail(50).features)
top50fea
### Got the top 50 predictive features

fea_coef

,coef,features
0,0.123648,MaxEStateIndex
1,-0.125100,MinEStateIndex
2,0.123915,MaxAbsEStateIndex
3,-0.092081,MinAbsEStateIndex
4,-0.175491,qed
...,...,...
199,0.137550,fr_thiazole
200,0.078924,fr_thiocyan
201,0.051207,fr_thiophene
202,0.214179,fr_unbrch_alkane


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import statsmodels.api as sm
# lst = []
lst = top50fea
# lst

In [12]:
correlated_features = set()
correlation_matrix = X.corr()

In [13]:
correlation_matrix.head(50)

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
MaxEStateIndex,1.000000,-0.443189,0.999918,-0.395368,0.059356,0.400759,0.404784,0.401557,0.447271,-0.043734,...,0.026018,0.135373,0.062720,0.022382,0.018507,-0.020118,-0.018527,0.001951,-0.079012,0.110018
MinEStateIndex,-0.443189,1.000000,-0.446904,0.021756,0.128365,-0.377098,-0.392597,-0.377483,-0.344058,-0.007889,...,0.012300,-0.354797,-0.132409,0.002933,-0.004028,0.004569,0.027278,0.007970,0.045676,-0.081912
MaxAbsEStateIndex,0.999918,-0.446904,1.000000,-0.394075,0.058520,0.401061,0.405069,0.401860,0.447638,-0.043762,...,0.025911,0.135245,0.062658,0.022332,0.018479,-0.020183,-0.018550,0.001898,-0.078267,0.109900
MinAbsEStateIndex,-0.395368,0.021756,-0.394075,1.000000,-0.089060,-0.206750,-0.202661,-0.206773,-0.225461,-0.019688,...,-0.013547,-0.045776,-0.020404,-0.021920,-0.020968,0.005988,0.012244,0.007601,0.018746,-0.023803
qed,0.059356,0.128365,0.058520,-0.089060,1.000000,-0.214616,-0.207920,-0.214931,-0.203042,-0.044628,...,-0.033946,0.102333,0.045041,0.031351,-0.048155,0.014141,-0.046595,0.034152,-0.340000,0.026420
MolWt,0.400759,-0.377098,0.401061,-0.206750,-0.214616,1.000000,0.998130,0.999992,0.940381,0.013249,...,0.079127,0.125962,0.026539,0.006168,0.045758,0.029302,-0.029739,0.061635,0.123410,0.039597
HeavyAtomMolWt,0.404784,-0.392597,0.405069,-0.202661,-0.207920,0.998130,1.000000,0.998024,0.923507,0.016883,...,0.080229,0.132257,0.029501,0.004410,0.048052,0.033520,-0.028441,0.063623,0.093397,0.043541
ExactMolWt,0.401557,-0.377483,0.401860,-0.206773,-0.214931,0.999992,0.998024,1.000000,0.941086,0.013264,...,0.079219,0.126089,0.026500,0.006193,0.045823,0.029346,-0.029720,0.061584,0.123736,0.039638
NumValenceElectrons,0.447271,-0.344058,0.447638,-0.225461,-0.203042,0.940381,0.923507,0.941086,1.000000,-0.023337,...,0.069907,0.115456,0.015426,0.014047,0.040963,0.016901,-0.034858,0.051274,0.188666,0.042289
NumRadicalElectrons,-0.043734,-0.007889,-0.043762,-0.019688,-0.044628,0.013249,0.016883,0.013264,-0.023337,1.000000,...,-0.008712,-0.009121,-0.004470,-0.003973,-0.002162,-0.005780,-0.002144,-0.004484,-0.014048,-0.008683


In [14]:
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

In [15]:
print(correlated_features)
print(len(correlated_features))

{'NumAromaticCarbocycles', 'Chi2n', 'Chi3v', 'ExactMolWt', 'fr_C_O_noCOO', 'fr_phenol_noOrthoHbond', 'MolMR', 'fr_Ar_N', 'Chi4n', 'fr_phos_ester', 'Chi4v', 'Chi1n', 'NumHAcceptors', 'Chi1v', 'NumSaturatedRings', 'fr_C_O', 'VSA_EState6', 'MinAbsPartialCharge', 'Chi0', 'fr_nitro_arom', 'VSA_EState10', 'BCUT2D_CHGLO', 'NOCount', 'SlogP_VSA11', 'NumSaturatedCarbocycles', 'BCUT2D_LOGPHI', 'SlogP_VSA6', 'NumHeteroatoms', 'HeavyAtomMolWt', 'FpDensityMorgan2', 'Chi1', 'NumAromaticRings', 'NumSaturatedHeterocycles', 'fr_COO', 'fr_phenol', 'fr_nitrile', 'Kappa1', 'fr_Nhpyrrole', 'NumHDonors', 'SlogP_VSA12', 'HeavyAtomCount', 'MaxAbsPartialCharge', 'fr_COO2', 'Chi0v', 'fr_Al_OH_noTert', 'fr_benzene', 'SlogP_VSA5', 'Chi3n', 'fr_ketone_Topliss', 'MaxAbsEStateIndex', 'Chi0n', 'LabuteASA', 'BertzCT', 'FpDensityMorgan3', 'Chi2v', 'NumValenceElectrons'}
56


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.drop(labels=correlated_features, axis=1, inplace=True)
X_test.drop(labels=correlated_features, axis=1, inplace=True)

In [17]:
numeric_columns=list(X_train.select_dtypes(['float64', 'int64']).columns)
categorical_columns=list(X_train.select_dtypes('object').columns)
print(len(numeric_columns))

pipeline=ColumnTransformer([
    ('num',StandardScaler(),numeric_columns),
    ('cat', 'passthrough', categorical_columns)])
new_X_train=pipeline.fit_transform(X_train)
new_X_train

148


array([[-1.3680955 ,  0.51529919, -0.70682065, ..., -0.08442397,
        -0.29546398, -0.15751235],
       [ 0.05643239,  0.49201771, -0.62114972, ..., -0.08442397,
        -0.29546398, -0.15751235],
       [ 0.4372603 , -0.05804124, -0.18138986, ..., -0.08442397,
         1.00341283, -0.15751235],
       ...,
       [-2.26201369,  0.86351946,  0.57455689, ..., -0.08442397,
        -0.29546398, -0.15751235],
       [ 0.39290113, -0.32923628, -0.70682065, ..., -0.08442397,
        -0.29546398, -0.15751235],
       [-0.52564169,  0.41404595, -0.33422998, ..., -0.08442397,
        -0.29546398, -0.15751235]])

In [18]:
X_train.corr().head(50)

,MaxEStateIndex,MinEStateIndex,MinAbsEStateIndex,qed,MolWt,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,FpDensityMorgan1,BCUT2D_MWHI,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
MaxEStateIndex,1.000000,-0.448062,-0.386773,0.061642,0.395303,-0.045999,0.323413,-0.196706,0.127145,-0.097775,...,0.018887,0.139979,0.061974,0.018541,0.009301,-0.018120,-0.017809,0.002697,-0.072783,0.108666
MinEStateIndex,-0.448062,1.000000,0.010521,0.120745,-0.378913,-0.005507,-0.457717,0.211329,0.043931,-0.106248,...,0.008606,-0.372410,-0.132329,0.002963,-0.003351,0.005230,0.028249,0.003688,0.046331,-0.084290
MinAbsEStateIndex,-0.386773,0.010521,1.000000,-0.092312,-0.204806,-0.019249,-0.203015,0.350273,-0.118170,-0.033865,...,-0.010656,-0.043156,-0.020820,-0.020132,-0.017961,0.003373,0.015428,0.001123,0.014062,-0.023748
qed,0.061642,0.120745,-0.092312,1.000000,-0.208109,-0.039877,-0.219913,0.105480,0.132220,-0.018640,...,-0.023352,0.098834,0.050064,0.029340,-0.039990,0.032389,-0.042968,0.044046,-0.342637,0.031377
MolWt,0.395303,-0.378913,-0.204806,-0.208109,1.000000,0.010717,0.248735,-0.258975,-0.431875,0.294685,...,0.066557,0.138183,0.023359,0.005050,0.038634,0.022151,-0.027102,0.054032,0.132919,0.037903
NumRadicalElectrons,-0.045999,-0.005507,-0.019249,-0.039877,0.010717,1.000000,0.249370,-0.052017,-0.046868,-0.002151,...,-0.007997,-0.008667,-0.004096,-0.003647,-0.001892,-0.005276,-0.001919,-0.004242,-0.012337,-0.007915
MaxPartialCharge,0.323413,-0.457717,-0.203015,-0.219913,0.248735,0.249370,1.000000,-0.395141,-0.004411,0.021562,...,0.020854,0.071474,-0.001133,-0.023744,0.047673,0.001840,0.005358,-0.009105,-0.039457,0.060767
MinPartialCharge,-0.196706,0.211329,0.350273,0.105480,-0.258975,-0.052017,-0.395141,1.000000,0.013975,-0.118891,...,0.025290,0.014543,0.038848,0.007675,-0.005678,0.010210,-0.005315,0.005160,-0.094901,0.045509
FpDensityMorgan1,0.127145,0.043931,-0.118170,0.132220,-0.431875,-0.046868,-0.004411,0.013975,1.000000,-0.012486,...,0.050342,-0.011073,-0.009756,0.044181,0.026896,0.076401,0.032960,0.019566,-0.327205,0.030724
BCUT2D_MWHI,-0.097775,-0.106248,-0.033865,-0.018640,0.294685,-0.002151,0.021562,-0.118891,-0.012486,1.000000,...,0.062280,0.065267,0.041021,-0.016098,0.001429,0.057362,0.011076,0.059946,-0.008835,0.007421


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import statsmodels.api as sm
log_reg = sm.Logit(y_train, new_X_train).fit(method='bfgs')
print(log_reg.summary())

         Current function value: 0.487282
         Iterations: 35
         Function evaluations: 37
         Gradient evaluations: 37
                           Logit Regression Results                           
Dep. Variable:        CHANNEL_OUTCOME   No. Observations:                 7063
Model:                          Logit   Df Residuals:                     6916
Method:                           MLE   Df Model:                          146
Date:                Thu, 08 Sep 2022   Pseudo R-squ.:                 -0.3861
Time:                        11:00:35   Log-Likelihood:                -3441.7
converged:                      False   LL-Null:                       -2482.9
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0397        nan        nan        nan         nan         n

/opt/anaconda3/envs/my-rdkit-env/lib/python3.10/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/opt/anaconda3/envs/my-rdkit-env/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [20]:
assay_2 = one_assay.copy()
assay_2 = assay_2.dropna()

class_0 = assay_2[assay_2['CHANNEL_OUTCOME'] == 'inactive'].sample(50)
class_1 = assay_2[assay_2['CHANNEL_OUTCOME'] == 'active antagonist'].sample(50)
small_sub2 = class_0.append(class_1)
small_sub2
# small_sub2['assay_name'] = 'tox21-ar-mda-kb2-luc-antagonist-p1'


X = small_sub2.iloc[:, 3:]
y = small_sub2['CHANNEL_OUTCOME']
y = y.replace(to_replace=['active antagonist', 'inactive'], value=[1, 0])

X = small_sub2[np.intersect1d(small_sub2.columns, lst)]

### only scale numeric columns
numeric_columns=list(X.select_dtypes(['float64', 'int64']).columns)
categorical_columns=list(X.select_dtypes('object').columns)
print(numeric_columns)

pipeline=ColumnTransformer([
    ('num',StandardScaler(),numeric_columns),
    ('cat', 'passthrough', categorical_columns)])

new_X=pipeline.fit_transform(X)
new_X

log_reg = sm.Logit(y, new_X).fit(method='bfgs')
print(log_reg.summary())

['BCUT2D_LOGPHI', 'BCUT2D_MRHI', 'BertzCT', 'Chi0v', 'Chi3v', 'Chi4n', 'EState_VSA5', 'EState_VSA9', 'ExactMolWt', 'HeavyAtomMolWt', 'MaxAbsEStateIndex', 'MaxAbsPartialCharge', 'MaxEStateIndex', 'MolWt', 'NumAliphaticHeterocycles', 'NumHAcceptors', 'PEOE_VSA14', 'PEOE_VSA8', 'SMR_VSA2', 'SMR_VSA4', 'SMR_VSA6', 'SlogP_VSA10', 'SlogP_VSA11', 'SlogP_VSA6', 'SlogP_VSA8', 'VSA_EState3', 'VSA_EState4', 'VSA_EState5', 'VSA_EState7', 'VSA_EState8', 'VSA_EState9', 'fr_ArN', 'fr_Ar_OH', 'fr_C_O_noCOO', 'fr_C_S', 'fr_NH0', 'fr_NH1', 'fr_aldehyde', 'fr_alkyl_halide', 'fr_azo', 'fr_furan', 'fr_halogen', 'fr_imide', 'fr_isocyan', 'fr_ketone', 'fr_nitro', 'fr_nitroso', 'fr_phenol', 'fr_thiazole', 'fr_unbrch_alkane']
         Current function value: 0.002015
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
                           Logit Regression Results                           
Dep. Variable:        CHANNEL_OUTCOME   No. Observations:                  1

/var/folders/mw/vsqjplld57v73jz2c0hw5x00llg6y6/T/ipykernel_54748/1902734200.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  small_sub2 = class_0.append(class_1)
/opt/anaconda3/envs/my-rdkit-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/opt/anaconda3/envs/my-rdkit-env/lib/python3.10/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/opt/anaconda3/envs/my-rdkit-env/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/envs/my-rdkit-env/lib/python3.10/site-packages/statsmodels/discrete/d

### Naive Bayes

In [22]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(class_weight='balanced', random_state=42, max_depth=20)

feat_selector = BorutaPy(
    verbose=2,
    estimator=rf,
    n_estimators='auto',
    max_iter=10  # number of iterations to perform
)

# train Boruta
# N.B.: X and y must be numpy arrays
feat_selector.fit(np.array(new_X_train), np.array(y_train))

# print support and ranking for each feature
print("\n------Support and Ranking for each feature------")
for i in range(len(feat_selector.support_)):
    if feat_selector.support_[i]:
        print("Passes the test: ", X_train.columns[i],
              " - Ranking: ", feat_selector.ranking_[i])

Iteration: 	1 / 10
Confirmed: 	0
Tentative: 	148
Rejected: 	0
Iteration: 	2 / 10
Confirmed: 	0
Tentative: 	148
Rejected: 	0
Iteration: 	3 / 10
Confirmed: 	0
Tentative: 	148
Rejected: 	0
Iteration: 	4 / 10
Confirmed: 	0
Tentative: 	148
Rejected: 	0
Iteration: 	5 / 10
Confirmed: 	0
Tentative: 	148
Rejected: 	0
Iteration: 	6 / 10
Confirmed: 	0
Tentative: 	148
Rejected: 	0
Iteration: 	7 / 10
Confirmed: 	0
Tentative: 	148
Rejected: 	0
Iteration: 	8 / 10
Confirmed: 	42
Tentative: 	17
Rejected: 	89
Iteration: 	9 / 10
Confirmed: 	42
Tentative: 	17
Rejected: 	89


BorutaPy finished running.

Iteration: 	10 / 10
Confirmed: 	42
Tentative: 	9
Rejected: 	89

------Support and Ranking for each feature------
Passes the test:  MaxEStateIndex  - Ranking:  1
Passes the test:  MinEStateIndex  - Ranking:  1
Passes the test:  MinAbsEStateIndex  - Ranking:  1
Passes the test:  qed  - Ranking:  1
Passes the test:  MolWt  - Ranking:  1
Passes the test:  MaxPartialCharge  - Ranking:  1
Passes the test:  MinPar

In [23]:
selected_rf_features = pd.DataFrame({'Feature':list(X_train.columns),
                                       'Ranking':feat_selector.ranking_})
selected_rf_features.sort_values(by='Ranking').head(40)

,Feature,Ranking
0,MaxEStateIndex,1
68,VSA_EState8,1
25,PEOE_VSA14,1
65,VSA_EState4,1
30,PEOE_VSA6,1
31,PEOE_VSA7,1
32,PEOE_VSA8,1
34,SMR_VSA1,1
35,SMR_VSA10,1
70,FractionCSP3,1


In [24]:
new_X_train.shape

(7063, 148)

In [25]:
class_weight = []
for i in range(len(y_train)):
    if y_train.values[i] == 1:
        class_weight.append(4.49)
    elif y_train.values[i] == 0:
        class_weight.append(0.56)
class_weight

[0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 4.49,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 4.49,
 0.56,
 4.49,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 4.49,
 0.56,
 0.56,
 0.56,
 4.49,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 4.49,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 4.49,
 0.56,
 0.56,
 0.56,
 4.49,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 4.49,
 0.56,
 0.56,
 0.56,
 4.49,
 0.56,
 0.56,
 0.56,
 0.56,
 4.49,
 4.49,
 0.56,
 0.56,
 4.49,
 0.56,
 0.56,
 4.49,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,
 0.56,

In [29]:
from sklearn.naive_bayes import BernoulliNB

X = one_assay.iloc[:,3:]
y = one_assay.CHANNEL_OUTCOME
y = y.replace(to_replace=['active antagonist', 'inactive'], value=[1, 0])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.drop(labels=correlated_features, axis=1, inplace=True)
X_test.drop(labels=correlated_features, axis=1, inplace=True)

scaler = StandardScaler()

new_X_train=scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)

X_important_train = feat_selector.transform(np.array(new_X_train))
X_important_test = feat_selector.transform(np.array(X_scaled_test)) 

clf = BernoulliNB()
# fit it
# clf.fit(X_train,y_train)
clf.fit(X_important_train,y_train, sample_weight= class_weight)
# test
y_pred = clf.predict(X_important_test)
yproba = clf.predict_proba(X_important_test)
# performance
print(f'\nBalanced Accuracy Score: {metrics.balanced_accuracy_score(y_test,y_pred)}')
print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'F1 score: {metrics.f1_score(y_test,y_pred)}')



Balanced Accuracy Score: 0.7008148543948705
Confusion Matrix: 
[[1056  520]
 [  51  139]]
Area Under Curve: 0.7008148543948705
F1 score: 0.32744405182567726


#### Draft

In [23]:
# Machine Learning Logistic Regression doesn't seem to work because of the negative R^2 value
# below shows unused code for maybe future use

# assay_2 = ar_mda_kb2_luc_antagonist_p1.copy()
# assay_2 = assay_2.dropna()

# class_0 = assay_2[assay_2['tox21-ar-mda-kb2-luc-antagonist-p1'] == 0].sample(50)
# class_1 = assay_2[assay_2['tox21-ar-mda-kb2-luc-antagonist-p1'] == 1].sample(50)
# small_sub2 = class_0.append(class_1)
# # small_sub


# X = small_sub2.iloc[:, 7:]
# y = small_sub2['tox21-ar-mda-kb2-luc-antagonist-p1']

# X = small_sub2[np.intersect1d(small_sub2.columns, lst)]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# # X

# numeric_columns=list(X_train.select_dtypes('float64').columns)
# categorical_columns=list(X_train.select_dtypes('int64').columns)
# print(numeric_columns)

# pipeline=ColumnTransformer([
#     ('num',StandardScaler(),numeric_columns),
#     ('cat', 'passthrough', categorical_columns)])

# new_X_train=pipeline.fit_transform(X_train)
# # new_X_train

# X_scaled_test = pipeline.transform(X_test)
# lg2 = LogisticRegression()
# # fit it
# lg2.fit(new_X_train,y_train)
# # test
# y_pred = lg2.predict(X_scaled_test)
# yproba = lg2.predict_proba(X_scaled_test)
# # performance
# print(f'\nBalanced Accuracy Score: {metrics.balanced_accuracy_score(y_test,y_pred)}')
# print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')
# print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
# print(f'F1 score: {metrics.f1_score(y_test,y_pred)}')
# print(f'R^2 score: {metrics.r2_score(y_test,y_pred)}')


In [26]:
y_train.value_counts()
# print(len(X_train.columns))
y_test.value_counts()

0    1576
1     190
Name: CHANNEL_OUTCOME, dtype: int64

In [27]:
X

,BCUT2D_LOGPHI,BCUT2D_MRHI,BertzCT,Chi0v,Chi3v,Chi4n,EState_VSA5,EState_VSA9,ExactMolWt,HeavyAtomMolWt,...,fr_furan,fr_halogen,fr_imide,fr_isocyan,fr_ketone,fr_nitro,fr_nitroso,fr_phenol,fr_thiazole,fr_unbrch_alkane
6742,2.311657,5.949042,808.251127,15.037158,4.224640,2.749588,24.395945,9.473726,395.145676,375.221,...,0,3,0,0,0,0,0,0,0,0
7210,2.027759,4.386659,25.019550,6.242641,1.457107,0.853553,38.524930,0.000000,114.140851,96.088,...,0,0,0,0,0,0,0,0,0,3
8182,2.079973,6.279505,253.335454,8.389748,1.446934,0.824429,0.000000,11.600940,201.078123,189.565,...,0,1,0,0,0,0,0,0,0,0
645,2.047429,5.865878,154.282097,4.755274,1.293389,0.799312,0.000000,5.106527,129.042593,122.059,...,0,0,0,0,0,0,0,0,0,0
4759,2.099471,5.663141,179.195441,9.781992,2.531284,1.613112,0.000000,10.213055,230.151809,208.128,...,0,0,0,0,0,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6849,2.349979,5.905803,475.525669,14.591012,4.036431,2.559739,44.945751,4.736863,338.209324,308.204,...,0,0,0,0,0,0,0,3,0,9
4120,2.359385,5.334292,331.606813,11.384286,4.769778,3.639785,32.104108,0.000000,262.168128,240.177,...,0,0,0,2,0,0,0,0,0,0
5200,2.696767,5.267581,1589.398675,36.340664,18.035195,14.737589,51.145706,28.421178,867.498021,794.487,...,0,0,0,0,0,0,0,0,0,0
4010,2.201349,6.255358,621.521023,16.352324,27.956988,2.516471,0.000000,17.025714,510.035480,496.084,...,0,2,0,0,0,0,0,0,0,0
